In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
from pathlib import Path

# ==============================================
# 1. Data Loading and Initial Inspection
# ==============================================

# Load data (replace with your file path)
df = pd.read_csv('solar_data.csv', parse_dates=['Timestamp'])

print("Initial Data Shape:", df.shape)
print("\nData Types and Missing Values:")
print(df.info())

# ==============================================
# 2. Summary Statistics & Missing Value Report
# ==============================================

print("\nSummary Statistics for Numeric Columns:")
print(df.describe())

# Missing value analysis
missing_values = df.isna().sum()
missing_pct = (missing_values / len(df)) * 100

print("\nMissing Value Report:")
missing_report = pd.DataFrame({
    'Missing_Count': missing_values,
    'Missing_Percentage': missing_pct
})
print(missing_report)

# List columns with >5% missing values
high_missing_cols = missing_pct[missing_pct > 5].index.tolist()
print("\nColumns with >5% missing values:", high_missing_cols)

# ==============================================
# 3. Outlier Detection & Basic Cleaning
# ==============================================

# Define key columns for analysis
sensor_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Z-score based outlier detection
z_scores = np.abs(stats.zscore(df[sensor_cols]))
outliers = (z_scores > 3).any(axis=1)
df['Outlier_Flag'] = outliers

print(f"\nFound {outliers.sum()} outlier rows (|Z| > 3)")

# Handle missing values - impute median for key columns
for col in sensor_cols:
    if col in df.columns:
        df[col].fillna(df[col].median(), inplace=True)

# Physical range validation
df = df[(df['GHI'] >= 0) & (df['GHI'] <= 1500)]  # Valid GHI range
df = df[df['WS'] >= 0]  # Wind speed can't be negative

# ==============================================
# 4. Export Cleaned Data
# ==============================================

# Create data directory if it doesn't exist
output_dir = Path('data')
output_dir.mkdir(exist_ok=True)

# Export cleaned data (country should be dynamic)
country = 'Germany'  # Change as needed
output_path = output_dir / f'{country}_clean.csv'
df.to_csv(output_path, index=False)
print(f"\nCleaned data saved to {output_path}")

# ==============================================
# 5. Time Series Analysis
# ==============================================

plt.figure(figsize=(15, 10))

# Plot GHI, DNI, DHI over time
plt.subplot(2, 1, 1)
df.set_index('Timestamp')['GHI'].plot(label='GHI')
df.set_index('Timestamp')['DNI'].plot(label='DNI')
df.set_index('Timestamp')['DHI'].plot(label='DHI')
plt.title('Solar Irradiance Over Time')
plt.ylabel('W/m²')
plt.legend()

# Plot Temperature over time
plt.subplot(2, 1, 2)
df.set_index('Timestamp')['Tamb'].plot(color='red')
plt.title('Ambient Temperature Over Time')
plt.ylabel('°C')

plt.tight_layout()
plt.show()

# Monthly patterns
df['Month'] = df['Timestamp'].dt.month
monthly_avg = df.groupby('Month')[['GHI', 'Tamb']].mean()

plt.figure(figsize=(10, 5))
monthly_avg['GHI'].plot(kind='bar', color='gold', alpha=0.7, label='GHI')
monthly_avg['Tamb'].plot(kind='line', color='red', secondary_y=True, label='Temperature')
plt.title('Monthly Average GHI and Temperature')
plt.ylabel('GHI (W/m²)')
plt.ylabel('Temperature (°C)', rotation=270, labelpad=15)
plt.legend()
plt.show()


# Compare sensor readings before/after cleaning
# (Assuming we have original and cleaned data)
# For demo, we'll use outlier flag as proxy

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='Outlier_Flag', y='ModA')
plt.title('ModA Readings: Outliers vs Clean Data')
plt.xlabel('Is Outlier')
plt.ylabel('ModA Value')
plt.show()



# Correlation heatmap
plt.figure(figsize=(10, 8))
corr_matrix = df[sensor_cols + ['Tamb', 'RH']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

# Scatter plots
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.scatterplot(data=df, x='WS', y='GHI', alpha=0.5)
plt.title('Wind Speed vs GHI')

plt.subplot(2, 2, 2)
sns.scatterplot(data=df, x='RH', y='Tamb', alpha=0.5)
plt.title('Relative Humidity vs Temperature')

plt.subplot(2, 2, 3)
sns.scatterplot(data=df, x='Tamb', y='GHI', alpha=0.5)
plt.title('Temperature vs GHI')

plt.tight_layout()
plt.show()


# Wind distribution histogram
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
df['WS'].hist(bins=30)
plt.title('Wind Speed Distribution')
plt.xlabel('Wind Speed (m/s)')

plt.subplot(1, 2, 2)
df['GHI'].hist(bins=30, color='gold')
plt.title('GHI Distribution')
plt.xlabel('GHI (W/m²)')
plt.show()


# 9. Temperature Analysis


plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Tamb', y='GHI', hue='RH', palette='viridis')
plt.title('GHI vs Temperature Colored by Relative Humidity')
plt.xlabel('Temperature (°C)')
plt.ylabel('GHI (W/m²)')
plt.colorbar(label='Relative Humidity (%)')
plt.show()

# 10. Bubble Chart


plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    x=df['Tamb'],
    y=df['GHI'],
    s=df['RH']*2,  # Bubble size based on RH
    c=df['WS'],    # Color based on wind speed
    alpha=0.6,
    cmap='viridis'
)
plt.colorbar(scatter, label='Wind Speed (m/s)')
plt.title('GHI vs Temperature (Bubble Size = RH, Color = WS)')
plt.xlabel('Temperature (°C)')
plt.ylabel('GHI (W/m²)')
plt.show()